# Investigating effective information in artificial neural networks

$$ EI = H(\langle W_i^\text{out} \rangle) - \langle H(W_i^\text{out}) \rangle $$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

In [93]:
def H(x, dim=0):
    """Compute the Shannon information entropy of x.
    
    Given a tensor x, compute the shannon entropy along one of its axes. If
    x.shape == (N,) then returns a scalar (0-d tensor). If x.shape == (N, N)
    then information can be computed along vertical or horizontal axes by
    passing arguments dim=0 and dim=1, respectively.
    
    Note that the function does not check that the axis along which
    information will be computed represents a valid probability distribution.
    
    Args:
        x (torch.tensor) containing probability distribution
        dim (int) dimension along which to compute entropy
    
    Returns:
        (torch.tensor) of a lower order than input x
    """
    r = x * torch.log2(x)
    r[r != r] = 0
    return -torch.sum(r, dim=dim)

In [82]:
def norm(W):
    """Turns 2x2 matrix W into a transition probability matrix.
    
    The weight/adjacency matrix of an ANN does not on its own allow for EI
    to be computed. This is because the out weights of a given neuron are not
    a probability distribution (they do not necessarily sum to 1). We therefore
    must normalize them. Here, I apply a softmax function to each row of matrix
    W to ensure that the out-weights are normalized.
    
    Args:
        W (torch.tensor) of shape (2, 2)
        
    Returns:
        (torch.tensor) of shape (2, 2)
    """
    return F.softmax(W, dim=1)

In [171]:
def EI(W, normalize=True):
    """Compute effective information from connectivity matrix W."""
    if normalize:
        W = norm(W)
    N = int(W.shape[0]) # number of neurons
    degeneracy = H(torch.sum(W, dim=0)/N)
    determinism = torch.mean(H(W, dim=1))
    return degeneracy - determinism

In [172]:
N = 100
EI(torch.normal(mean=torch.zeros((N, N)), std=1))

tensor(0.6555)

In [173]:
N = 100
EI(torch.rand((N,N)))

tensor(0.0571)

In [66]:
a = torch.zeros((5,))
a[2] = 1

In [75]:
H(W_norm, dim=1)

tensor([1.1222, 1.1524, 1.4886])